In [1]:
import sys
from pathlib import Path

# 현재 폴더: ptta/other_method/ActMAD/
# ptta 바로 위의 디렉토리를 sys.path에 추가
PROJECT_PARENT = Path.cwd().parents[0]  # -> ptta/ 의 부모 디렉토리
print(PROJECT_PARENT)
sys.path.insert(0, str(PROJECT_PARENT))

from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection, SHIFTDiscreteSubsetForObjectDetection

from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

# import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

/workspace/ptta


## Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 2
ADDITIONAL_GPU = 0
DATA_TYPE = torch.bfloat16

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

In [ ]:
PROJECT_NAME = "ActMAD test"
RUN_NAME = "RT-DETR_R50_ActMAD"

## Baseline

In [ ]:
import baseline

data_root = path.normpath(path.join(Path.cwd(), "..", "data"))
print(data_root)
direct_method = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./direct_method_checkpoints",
    method="direct_method",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=32,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

actmad = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./actmad_checkpoints",
    method="actmad",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

norm = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./norm_checkpoints",
    method="norm",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./dua_checkpoints",
    method="dua",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=16,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

mean_teacher = dua = baseline.Baseline(
    device=device,
    data_root=data_root,
    save_dir="./mean_teacher_checkpoints",
    method="mean_teacher",
    model_arch="rt_detr",
    reference_model_id="PekingU/rtdetr_r50vd",
    image_size=800,
    class_num=6,
    valid_batch=4,
    clean_bn_extract_batch=16,
    model_states="/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors",
    lr=1e-4,
    momentum=0.05,
    mom_pre=0.07,
    decay_factor=0.97,
    min_momentum_constant=0.01,
    use_scheduler=False,
    warmup_steps=50,
    decay_total_steps=20,
    patience=10,
    eval_every=10,
    enable_log_file=True
)

In [ ]:
mean_teacher.train()

In [ ]:
direct_method.train()

In [ ]:
actmad.train()

In [ ]:
norm.train()

In [ ]:
dua.train()

In [2]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat
from safetensors.torch import load_file

import utils

reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = 6

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 800, "width": 1280}
reference_preprocessor.do_resize = False

model_pretrained = RTDetrForObjectDetection(config=reference_config)
model_states = load_file("/workspace/ptta/RT-DETR_R50vd_SHIFT_CLEAR.safetensors", device="cpu")
model_pretrained.load_state_dict(model_states, strict=False)

for param in model_pretrained.parameters():
    param.requires_grad = False  # Freeze

# Initialize Model
model_pretrained

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [4]:
model, optimizer = utils.add_adapters(model_pretrained, device = None, reduction_ratio=32, target_stages=[0, 1, 2, 3])

Added adapter to stage 0, block 0, layer 0
Added adapter to stage 0, block 1, layer 0
Added adapter to stage 0, block 2, layer 0
Added adapter to stage 1, block 0, layer 0
Added adapter to stage 1, block 1, layer 0
Added adapter to stage 1, block 2, layer 0
Added adapter to stage 1, block 3, layer 0
Added adapter to stage 2, block 0, layer 0
Added adapter to stage 2, block 1, layer 0
Added adapter to stage 2, block 2, layer 0
Added adapter to stage 2, block 3, layer 0
Added adapter to stage 2, block 4, layer 0
Added adapter to stage 2, block 5, layer 0
Added adapter to stage 3, block 0, layer 0
Added adapter to stage 3, block 1, layer 0
Added adapter to stage 3, block 2, layer 0
Added 16 adapters to backbone
Total parameters: 42,925,340
Trainable parameters: 186,038
Trainable ratio: 0.0043


In [ ]:
for n,m in model.named_modules():
    if m.__class__.__name__ == "conv_task":
        print(n, hasattr(m, "down_proj"), hasattr(m, "up_proj"), getattr(m, "is_proj", None))
        params = [pn for pn,p in m.named_parameters() if p.requires_grad]
        print("trainable:", params)

In [6]:
import torch
from collections import OrderedDict

path = "/workspace/ptta/faster_rcnn_r50_shift.pth"   # cfg.TEST.ADAPTATION.SOURCE_FEATS_PATH
s = torch.load(path, map_location="cpu")

# 1) 최상위 키 확인
print("[top-level keys]:", list(s.keys()))

# 2) kl_div 존재/타입 확인
kl = s.get("kl_div", None)
print("has kl_div?", kl is not None, "type:", type(kl))

# # 3) kl_div가 dict인 경우: 키/샘플 값 미리보기
# if isinstance(kl, dict):
#     print("kl_div keys (len={}):".format(len(kl)), list(kl.keys())[:10])
#     # 값이 텐서일 수 있어요 → 파이썬 숫자로 변환해 미리보기
#     sample_items = list(kl.items())[:10]
#     preview = {k: (v.item() if hasattr(v, "item") else float(v) if isinstance(v, (int,float)) else type(v).__name__)
#                for k, v in sample_items}
#     print("kl_div preview:", preview)

# # 4) 텐서/리스트 등 다른 형태 대비
# elif hasattr(kl, "shape"):  # torch.Tensor
#     print("kl_div tensor shape:", kl.shape)
#     print("first few:", kl.flatten()[:10])
# elif isinstance(kl, list):
#     print("kl_div list len:", len(kl), "first few:", kl[:10])

# # 5) 전체 합/통계 (dict[...]->scalar 값 가정)
# def to_scalar(x):
#     if hasattr(x, "item"):
#         return x.item()
#     return float(x)

# if isinstance(kl, dict):
#     vals = [to_scalar(v) for v in kl.values()]
#     print("sum:", sum(vals), "mean:", sum(vals)/len(vals) if vals else None)

[top-level keys]: ['model', 'trainer', 'iteration']
has kl_div? False type: <class 'NoneType'>


In [5]:
print(model.s_div.values())

AttributeError: 'RTDetrForObjectDetection' object has no attribute 's_div'